In [1]:
import sys
sys.path.append("../")

from bayesflow import Approximator
from bayesflow import OfflineDataset
from bayesflow.simulators import TwoMoonsSimulator
from bayesflow.networks import CIF

In [2]:
batch_size = 128
num_batches = 32

simulator = TwoMoonsSimulator()
data = simulator.sample((batch_size * num_batches,))
dataset = OfflineDataset(data, workers=4, batch_size=batch_size)
print("Batch size:", dataset.batch_size)
print([key for key in dataset[0].keys()])


Batch size: 128
['r', 'alpha', 'theta', 'x']


In [3]:
cif = CIF()
approximator = Approximator(
	inference_network=cif,
	inference_variables=["theta"],
	inference_conditions=["r", "alpha", "x"]
)
approximator.compile(optimizer="adam")
approximator.build_from_data(next(iter(dataset)))

In [4]:
approximator.fit(dataset, epochs=5)

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 61s 110ms/step - loss: nan
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - loss: nan
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - loss: nan
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - loss: nan
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - loss: nan


In [5]:
samples = approximator.sample((128,2), next(iter(dataset)))

InvalidArgumentError: Exception encountered when calling SingleCoupling.call().

[1m{{function_node __wrapped__ConcatV2_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ConcatOp : Ranks of all input tensors should match: shape[0] = [128,2,1] vs. shape[1] = [128,2,2,2] [Op:ConcatV2] name: concat[0m

Arguments received by SingleCoupling.call():
  • x1=tf.Tensor(shape=(128, 2, 1), dtype=float32)
  • x2=tf.Tensor(shape=(128, 2, 1), dtype=float32)
  • conditions=tf.Tensor(shape=(128, 2, 2, 2), dtype=float32)
  • inverse=True
  • kwargs=<class 'inspect._empty'>